In [ ]:
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
import numpy as np


In [ ]:

ettH1_data = pd.read_csv('ETTh1.csv')
ettH2_data = pd.read_csv('ETTh2.csv')
ettM1_data = pd.read_csv('ETTm1.csv')
ettM2_data = pd.read_csv('ETTm2.csv')


etth1_data = ettH1_data[['date', 'HUFL']]
etth1_data['date'] = pd.to_datetime(etth1_data['date'])
etth1_data.set_index('date', inplace=True)
etth1_values = etth1_data.values.reshape(-1, 1).astype(float)

etth2_data = ettH2_data[['date', 'HULL']]
etth2_data['date'] = pd.to_datetime(etth2_data['date'])
etth2_data.set_index('date', inplace=True)
etth2_values = etth2_data.values.reshape(-1, 1).astype(float)

ettm1_data = ettM1_data[['date', 'HUFL']]
ettm1_data['date'] = pd.to_datetime(ettm1_data['date'])
ettm1_data.set_index('date', inplace=True)
ettm1_values = ettm1_data.values.reshape(-1, 1).astype(float)

ettm2_data = ettM2_data[['date', 'HULL']]
ettm2_data['date'] = pd.to_datetime(ettm2_data['date'])
ettm2_data.set_index('date', inplace=True)
ettm2_values = ettm2_data.values.reshape(-1, 1).astype(float)


scaler_etth1 = MinMaxScaler()
scaler_etth2 = MinMaxScaler()
scaler_ettm1 = MinMaxScaler()
scaler_ettm2 = MinMaxScaler()

etth1_scaled = scaler_etth1.fit_transform(etth1_values)
etth2_scaled = scaler_etth2.fit_transform(etth2_values)
ettm1_scaled = scaler_ettm1.fit_transform(ettm1_values)
ettm2_scaled = scaler_ettm2.fit_transform(ettm2_values)


C:\Users\ashis\AppData\Local\Temp\ipykernel_29288\3746508040.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  etth1_data['date'] = pd.to_datetime(etth1_data['date'])
C:\Users\ashis\AppData\Local\Temp\ipykernel_29288\3746508040.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  etth2_data['date'] = pd.to_datetime(etth2_data['date'])
C:\Users\ashis\AppData\Local\Temp\ipykernel_29288\3746508040.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .lo

In [ ]:
def create_lstm_model(sequence_length):
    model = Sequential()
    model.add(LSTM(units=50, return_sequences=True, input_shape=(sequence_length, 1)))
    model.add(LSTM(units=50))
    model.add(Dense(units=1))
    model.compile(optimizer='adam', loss='mean_squared_error')
    return model

def train_lstm(data):
    sequence_length = 50
    X = []
    y = []
    for i in range(len(data) - sequence_length):
        X.append(data[i:(i + sequence_length), 0])
        y.append(data[i + sequence_length, 0])
    X, y = np.array(X), np.array(y)

    X = np.reshape(X, (X.shape[0], X.shape[1], 1))
    model = create_lstm_model(sequence_length)
    model.fit(X, y, epochs=5, batch_size=32)

    predictions = []
    current_sequence = X[-1]
    for i in range(10 * sequence_length):
        prediction = model.predict(current_sequence.reshape(1, sequence_length, 1))[0, 0]
        predictions.append(prediction)
        current_sequence = np.roll(current_sequence, -1)
        current_sequence[-1] = prediction

    return np.array(predictions)



predictions_ETTh1 = train_lstm(etth1_scaled)
predictions_ETTh2 = train_lstm(etth2_scaled)
predictions_ETTm1 = train_lstm(ettm1_scaled)
predictions_ETTm2 = train_lstm(ettm2_scaled)

combined_matrix = np.column_stack((predictions_ETTh1, predictions_ETTh2, predictions_ETTm1, predictions_ETTm2))
print(combined_matrix)



Epoch 1/5
543/543 [==============================] - 25s 38ms/step - loss: 0.0125
Epoch 2/5
543/543 [==============================] - 21s 39ms/step - loss: 0.0038
Epoch 3/5
543/543 [==============================] - 21s 39ms/step - loss: 0.0030
Epoch 4/5
543/543 [==============================] - 22s 41ms/step - loss: 0.0028
Epoch 5/5
1/1 [==============================] - 0s 51ms/step
Epoch 1/5
543/543 [==============================] - 36s 49ms/step - loss: 0.0030
Epoch 2/5
543/543 [==============================] - 26s 49ms/step - loss: 0.0013
Epoch 3/5
543/543 [==============================] - 25s 46ms/step - loss: 0.0011
Epoch 4/5
543/543 [==============================] - 24s 44ms/step - loss: 0.0010
Epoch 5/5
1/1 [==============================] - 0s 39ms/step
Epoch 1/5
2176/2176 [==============================] - 109s 46ms/step - loss: 0.0028
Epoch 2/5
2176/2176 [==============================] - 102s 47ms/step - loss: 0.0011
Epoch 3/5
2176/2176 [=============================

In [ ]:
len_ETTh1 = len(predictions_ETTh1)
len_ETTh2 = len(predictions_ETTh2)
len_ETTm1 = len(predictions_ETTm1)
len_ETTm2 = len(predictions_ETTm2)

print(len_ETTh1, len_ETTh2, len_ETTm1, len_ETTm2)


500 500 500 500


In [ ]:
import pandas as pd

date_range = pd.date_range(start='2023-01-01', periods=len_ETTh1, freq='H')

combined_matrix = pd.DataFrame({
    'Date': date_range,
    'Predictions_ETTh1': predictions_ETTh1,
    'Predictions_ETTh2': predictions_ETTh2,
    'Predictions_ETTm1': predictions_ETTm1,
    'Predictions_ETTm2': predictions_ETTm2
})

print(combined_matrix)


                   Date  Predictions_ETTh1  Predictions_ETTh2  \
0   2023-01-01 00:00:00           0.721570           0.560671   
1   2023-01-01 01:00:00           0.754666           0.564047   
2   2023-01-01 02:00:00           0.776955           0.565802   
3   2023-01-01 03:00:00           0.792619           0.567168   
4   2023-01-01 04:00:00           0.806260           0.568374   
..                  ...                ...                ...   
495 2023-01-21 15:00:00           0.866130           0.787309   
496 2023-01-21 16:00:00           0.869068           0.787328   
497 2023-01-21 17:00:00           0.856256           0.787347   
498 2023-01-21 18:00:00           0.819630           0.787366   
499 2023-01-21 19:00:00           0.753085           0.787384   

     Predictions_ETTm1  Predictions_ETTm2  
0             0.734121           0.618732  
1             0.732415           0.617975  
2             0.729049           0.616806  
3             0.722603           0.615449  